Problème : analyser les transcriptions de vidéos afin de déterminer les différents sujets abordés dans les vidéos.

Problèmes : 
* 1- comment déterminer un tags pour une vidéo?
* 2- nos transcriptions ne sont pas labialisées. Comment labelliser de nombreuse transcriptions ? 


Solutions proposées: 
1- 
* La déterminations des sujet principaux dans un texte peut être faite avec de nombreuses techniques.
Celles que nous avons retenues sont :
* Latent Derichelet technique []
* TF-IDF
L'algorithme du TF-IDF identifie les mots les plus important dans le texte en fonction de leur fréquence.
Dans le texte. Contrairement au LDA il ne permet pas d'associer les mots à un topic.

Le LDA est expliqué dans ce Kernel nous avons réalisé un petit exemple sur un petit set de 3 transcriptions de 
vidéos prise sur le youtube easy-movie.

L'algorithme du LDA permet pour un K donné de déterminer une K topics dans un texte en utilisant.
les probabilité conditionnel. L'algorithme ne donne pas le nom des topics, il associe à chaque topic.
une les mots qui ont la plus forte probabilité de générer le topic.

Avec cette algorithme, nous pourrons obtenir une liste de labels pour les transcriptions
Par la suite, nous pourrions à l'aide de cette liste de labels déterminer le thème principal
du document. Il nous sera donc possible de labelliser avec un tag unique chaque transcription.
 
2 - 
Afin de tagger un grand nombre de transcriptions, nous avons pensé a construire un outil de 
labélisation de texte. Cette Outils permettra à un utilisateur d'un simple clique de 
tagger les textes rapidement.

* une interface graphique affichera le texte ainsi que les K (paramètre réglable) mots associé
a la topic.
* L'utilisateur aura alors la possibilité de tagger le texte avec des tags prédéfini ou de simplement
créer un nouveau tag et l'associé à la transcription.
* Une fois les textes labellisé il sera possible d'exporter un fichier CSV (ou autre format) pour
entrainé des modèles sur les données labélisées.

Afin de classifier les textes par topiques 4 modèles ont été retenu :

Scikit-learn:
* Linear-SVR
* KNN
* Logistic regression
Deep-learning Pytorch :
* Classification with attention based netwxork with BLSTM


Ces modèles ont été choisis en fonction de la nature du problème, la classification de donner en K
catégories et également pour leurs précisions sur ce genre de problèmes. 
 Ressource : 
 
 Comparaison de model :
 https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568 
 
 Attention networks:
 https://humboldt-wi.github.io/blog/research/information_systems_1819/group5_han/
 https://towardsdatascience.com/attention-based-neural-machine-translation-b5d129742e2c
      

In [36]:
from sklearn.decomposition  import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 



        
        




In [37]:
from os import listdir
from os.path import isfile, join
import pandas as pd 
import pysrt

class LoadData:
    __df = pd.DataFrame()

    def __init__(self, path, extension='srt'):
        self.path = path 
        self.__files = [f for f in listdir(self.path) if (isfile(join(self.path, f)) and f.endswith(extension))]
    
    def files(self):
        return self.__files
    
    def data(self, verbose=True):
        textes = []
        for file in self.__files:
            if verbose :
                print(f"reading info of file {file}")
            texte = self.__treat_srt_file(file, verbose=verbose)
            textes.append(texte)
        self.__df['title'] = self.files()
        self.__df['texte'] = textes
        return self.__df
        
    def __treat_srt_file(self, file, verbose):
        f_path = "{}{}".format(self.path, file)
        print(f_path)
        f = pysrt.open(f_path, encoding='utf-8')
        article = ""
        for t in f:
            article += " " + t.text
        print(f"article {article}")
        return article
            
        
        

In [38]:
loader = LoadData(path='./TextFiles')

for file in loader.files():
    print(file)
    

saint-gobin.srt
supbiotect.srt
evo-comment-utiliser-easy-movie.srt
scor-simplifier-harmoniser.srt


In [39]:
df = loader.data()
df

reading info of file saint-gobin.srt
./saint-gobin.srt
article  [Musique] bonjour je suis séverine du service communication de saint gobain distribution bâtiment france et je vais vous présenter izimmo ville vous souhaitez communiquer avec des vidéos nous vous proposons la solution easy movie pour vous y aider simple pratique et rapide l'outil imovie permet de faire des vidéos courtes à moindre coût et de qualité comment fonctionne izimmo vient créer un nouveau projet sélectionnez votre scénario choisissez votre plan est filmée pour utiliser imovie c'est très simple il vous suffit de suivre les instructions sur l'appli imovie et de lancer l'enregistrement une fois votre film terminé il vous suffit de l'envoyer aux équipes de montage du movie 48 heures plus tard vous recevez votre vidéo montée si cette solution vous intéresse n'hésitez pas à nous contacter à bientôt [Musique]
reading info of file supbiotect.srt
./supbiotect.srt
article  bonjour je suis louis julian ancien de sup'biotech

,title,texte
0,saint-gobin.srt,[Musique] bonjour je suis séverine du service...
1,supbiotect.srt,bonjour je suis louis julian ancien de sup'bi...
2,evo-comment-utiliser-easy-movie.srt,on peut y aller là le cadre il est bon voilà ...
3,scor-simplifier-harmoniser.srt,c'était global avec 38 bureaux implantés part...


CountVectorizer:

Convert a collection of text documents to a matrix of token counts


If you do not provide an a-priori dictionary and you do not use an analyzer that does some kind of feature 
selection then the number of features will be equal to the vocabulary size found by analyzing the data.

* max_df : If a word appear in more than 90% of the document it will be discarded 
* min_df : If a word appear less than 2 time it will not be take in count 

I used spacy french stop words list. We need to remove all the french 
stop word from our token because they appear in all the document and 
can be considered as topic's words. 

In [40]:
import spacy
from spacy.lang.fr.examples import sentences 
import spacy.cli
spacy.cli.download("fr_core_news_sm")
spacy_nlp = spacy.load('fr_core_news_sm')

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [41]:
spacy_stop_words = spacy.lang.fr.stop_words.STOP_WORDS

for i, w in enumerate(spacy_stop_words):
    print(w)
    if i >= 50:
        break 

nous
paf
ça
mes
la
mince
celle
toi
unes
parmi
crac
en
font
lesquels
tiennes
contre
sein
vingt
quoique
celles
dring
moi-meme
vous
soi-même
reste
minimale
devers
ès
merci
suivre
ouste
depuis
hurrah
meme
suivantes
aupres
chaque
olé
près
euh
extenso
treize
anterieur
j’
rend
autrement
possessif
quant
trois
different
d’


In [42]:
count_vectorizer = CountVectorizer(stop_words=spacy_stop_words)

In [43]:
dtm = count_vectorizer.fit_transform(df['texte'])

/Users/remy.d.w/.virtualenvs/PROROK/untitled/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


LDA use the naives bayes probability to affect each word to a 
topic and text to a topic.

* each word is associated to a topic. This topic is the one that have the 
best chance to  generate the word in this document. Example the word 'formula one'
 have higher probabilty to be generated in a sport or car document that in a cooking one.
 
* The new theme is the one that have the best chance to generate the document.

* P(T | D) give the likelihood that the Doc D is affected to the Theme T  
* P(W | T) give the likelihood that the Word W is affected to the Theme T

Here we want to find the 10 topic. For each topic a list of word are choosen.

In [44]:
LDA = LatentDirichletAllocation(n_components=10)

LDA.fit(dtm)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [45]:
for i, topic in enumerate(LDA.components_):
    print("\n\n\n\n")
    print(f"Topic n° {i}")
    last_ten_words = topic.argsort()[-5:]
    features = count_vectorizer.get_feature_names()
    for index in last_ten_words:
        print(features[index])






Topic n° 0
outil
solution
cadre
vidéos
musique





Topic n° 1
izimmo
simple
solution
musique
imovie





Topic n° 2
ensemble
dynamique
paris
culture
vidéos





Topic n° 3
outil
solution
cadre
vidéos
musique





Topic n° 4
outil
solution
cadre
vidéos
musique





Topic n° 5
outil
solution
cadre
vidéos
musique





Topic n° 6
crois
allez
cadre
mal
mettre





Topic n° 7
outil
solution
cadre
vidéos
musique





Topic n° 8
outil
solution
cadre
vidéos
musique





Topic n° 9
eau
hydroponie
toits
kg
vraiment


In [46]:
supbiotech_vec = count_vectorizer.transform([df.iloc[0, 1]])

In [47]:
saint_gobin_topics = LDA.transform(supbiotech_vec)

def extractTopic(components):
    last_ten_words = components.argsort()[-5:]
    features = count_vectorizer.get_feature_names()
    for index in last_ten_words:
        print(features[index])

In [48]:
index = np.argmax(saint_gobin_topics)
extractTopic(LDA.components_[index])

izimmo
simple
solution
musique
imovie


We are going to add each labels array to his corresponding text.

In [49]:
def extractTopicAndLoadInDataframe(count_vect, dataFrame, lda, nb_words=5):
    """
    
    :param cv: countVectorizer that was applied on the documents 
    :param df: the dataframe that contains the data 
    :param lda: Latent derichlet allocation trained 
    :param nb_words: number of words by topic that you want to used
    :return: 
    """
    document_labels = []
    for i, text in enumerate(dataFrame['texte']):
        text_vectorized = count_vect.transform([text])
        index = np.argmax(lda.transform(text_vectorized))
        topic = lda.components_[index]
        topic_word_index = topic.argsort()[-nb_words:] # get the 5 best word of the topic
        features = count_vect.get_feature_names()
        labels = []
        for j in topic_word_index:
            labels.append(features[j]) 
        document_labels.append(labels)
    dataFrame['labels'] = document_labels
        

In [50]:
extractTopicAndLoadInDataframe(count_vect=count_vectorizer,dataFrame=df, lda=LDA, nb_words=5)

In [51]:
df


,title,texte,labels
0,saint-gobin.srt,[Musique] bonjour je suis séverine du service...,"[izimmo, simple, solution, musique, imovie]"
1,supbiotect.srt,bonjour je suis louis julian ancien de sup'bi...,"[eau, hydroponie, toits, kg, vraiment]"
2,evo-comment-utiliser-easy-movie.srt,on peut y aller là le cadre il est bon voilà ...,"[crois, allez, cadre, mal, mettre]"
3,scor-simplifier-harmoniser.srt,c'était global avec 38 bureaux implantés part...,"[ensemble, dynamique, paris, culture, vidéos]"


I build a pipline. This pipline contains:
 * the cleaning of the data
   * lemming it's a normalization technique which converts high dimensional features into low dimensional features.
       We simply identify prefix and suffix and produce the simplest version of the word. 
       For example : connect, connection, connected => have different grammatical function but for ML models we need
       that this words be one simple word (connect) because it's can be confusing.
       
   * tokenizing : transform into token 
   * removing punct : It's useless for text classification 
   * removing stop words : stop words can lead to poor performance. 
  
 * Vectorize the data with CountVectorizer: Convert a collection of text documents to a matrix of token counts
 * Apply the LDA on the data : have been previously explained. 

In [52]:
from sklearn.base import BaseEstimator, TransformerMixin

import spacy.cli
from spacy.lang.fr import French
spacy.cli.download("fr_core_news_sm")
spacy_nlp = spacy.load('fr_core_news_sm')

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [53]:
parser = French()

tokenizer = French()

stop_words = spacy.lang.fr.stop_words.STOP_WORDS

punctuations = spacy.lang.fr.punctuation.LIST_PUNCT

def easy_tokenizer(sentence):
    """
    
    :param sentence: the texte we want to treat with our parser 
    :return: 
    """
    # creating our token object 
    easy_tokens = parser(sentence)
    
    # lemming  and lowering. Stripping white space 
    # if the word is not a Pronoun lower the word and strip space else return the lower word. 
    # Because  spacy replace pronouns bu "-PRON-" on lemming call. as we need the pronoun we can not do its lemmatization
    
    easy_tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower() for word in easy_tokens ]
    
    print(easy_tokens)
    
    # remove stop words 
    easy_tokens = [word  for word in easy_tokens if (word not in stop_words and word not in punctuations)]

    
    return easy_tokens

In [54]:
# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

class PrepareText(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X):
        """
        Take a dataframe as paremeter and fit to the data set è
        :param X: 
        :return: 
        """
        return self
    
    def transform(self, X):
        """
        remove stop word in the text and retore the text
        return a data frame that contains all the text 
        :param X: 
        :return: 
        """
        return [clean_text(text) for text in X]
    

In [55]:
easy_vectorizer = CountVectorizer(tokenizer=easy_tokenizer, ngram_range=(1,1))

In [56]:


from sklearn.pipeline import Pipeline
easy_lda = LatentDirichletAllocation(n_components=5)
X = df['texte']
easy_pipeline = Pipeline([('vectorizer', easy_vectorizer),("topic-analyser", easy_lda)])
easy_pipeline.fit(X=X)
topic_analysis = easy_pipeline.transform(X=X)

['', '[', 'musique', ']', 'bonjour', 'je', 'suis', 'séverine', 'du', 'service', 'communication', 'de', 'saint', 'gobain', 'distribution', 'bâtiment', 'france', 'et', 'je', 'vais', 'vous', 'présenter', 'izimmo', 'ville', 'vous', 'souhaitez', 'communiquer', 'avec', 'des', 'vidéos', 'nous', 'vous', 'proposons', 'la', 'solution', 'easy', 'movie', 'pour', 'vous', 'y', 'aider', 'simple', 'pratique', 'et', 'rapide', "l'", 'outil', 'imovie', 'permet', 'de', 'faire', 'des', 'vidéos', 'courtes', 'à', 'moindre', 'coût', 'et', 'de', 'qualité', 'comment', 'fonctionne', 'izimmo', 'vient', 'créer', 'un', 'nouveau', 'projet', 'sélectionnez', 'votre', 'scénario', 'choisissez', 'votre', 'plan', 'est', 'filmée', 'pour', 'utiliser', 'imovie', "c'", 'est', 'très', 'simple', 'il', 'vous', 'suffit', 'de', 'suivre', 'les', 'instructions', 'sur', "l'", 'appli', 'imovie', 'et', 'de', 'lancer', "l'", 'enregistrement', 'une', 'fois', 'votre', 'film', 'terminé', 'il', 'vous', 'suffit', 'de', "l'", 'envoyer', 'aux'

In [58]:
extractTopicAndLoadInDataframe(count_vect=easy_vectorizer,dataFrame=df, lda=easy_lda, nb_words=5)

['', '[', 'musique', ']', 'bonjour', 'je', 'suis', 'séverine', 'du', 'service', 'communication', 'de', 'saint', 'gobain', 'distribution', 'bâtiment', 'france', 'et', 'je', 'vais', 'vous', 'présenter', 'izimmo', 'ville', 'vous', 'souhaitez', 'communiquer', 'avec', 'des', 'vidéos', 'nous', 'vous', 'proposons', 'la', 'solution', 'easy', 'movie', 'pour', 'vous', 'y', 'aider', 'simple', 'pratique', 'et', 'rapide', "l'", 'outil', 'imovie', 'permet', 'de', 'faire', 'des', 'vidéos', 'courtes', 'à', 'moindre', 'coût', 'et', 'de', 'qualité', 'comment', 'fonctionne', 'izimmo', 'vient', 'créer', 'un', 'nouveau', 'projet', 'sélectionnez', 'votre', 'scénario', 'choisissez', 'votre', 'plan', 'est', 'filmée', 'pour', 'utiliser', 'imovie', "c'", 'est', 'très', 'simple', 'il', 'vous', 'suffit', 'de', 'suivre', 'les', 'instructions', 'sur', "l'", 'appli', 'imovie', 'et', 'de', 'lancer', "l'", 'enregistrement', 'une', 'fois', 'votre', 'film', 'terminé', 'il', 'vous', 'suffit', 'de', "l'", 'envoyer', 'aux'

In [59]:
df






,title,texte,labels
0,saint-gobin.srt,[Musique] bonjour je suis séverine du service...,"[solution, musique, ], [, imovie]"
1,supbiotect.srt,bonjour je suis louis julian ancien de sup'bi...,"[toits, terre, hydroponie, eau, vraiment]"
2,evo-comment-utiliser-easy-movie.srt,on peut y aller là le cadre il est bon voilà ...,"[préparez, sons, , mal, mettre]"
3,scor-simplifier-harmoniser.srt,c'était global avec 38 bureaux implantés part...,"[dynamique, ensemble, groupe, culture, vidéos]"
